In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('airbnb_hw17.db')

In [3]:
hosts = '''
CREATE TABLE IF NOT EXISTS hosts(
   id serial PRIMARY KEY,
   name VARCHAR(50) NOT NULL,
   contacts VARCHAR(50) NOT NULL
);'''

guests = '''
CREATE TABLE IF NOT EXISTS guests(
   id serial PRIMARY KEY,
   name VARCHAR(50) NOT NULL,
   contacts VARCHAR(50) NOT NULL
);
'''

rooms = '''
CREATE TABLE IF NOT EXISTS rooms(
   id serial PRIMARY KEY,
   host_id INT NOT NULL,
   title VARCHAR(50) NOT NULL,
   description TEXT,
   amount_guest INT CHECK (amount_guest > 0),
   price MONEY NOT NULL,
   location VARCHAR(100) NOT NULL,
   air_conditioner BOOLEAN,

   FOREIGN KEY (host_id)
       REFERENCES hosts (id)
);'''

host_reviews = '''
CREATE TABLE IF NOT EXISTS host_reviews(
   id serial PRIMARY KEY,
   description TEXT,
   rating INT CHECK (rating >= 0 AND rating <= 10),
   room_id INT NOT NULL,
   host_id INT NOT NULL,
   guest_id INT NOT NULL,
   review_date TIMESTAMP NOT NULL,
   
   FOREIGN KEY (room_id)
       REFERENCES rooms (id),
   FOREIGN KEY (host_id)
       REFERENCES hosts (id),
   FOREIGN KEY (guest_id)
       REFERENCES guests (id)
);'''

guest_reviews = '''
CREATE TABLE IF NOT EXISTS guest_reviews(
   id serial PRIMARY KEY,
   description TEXT,
   rating INT CHECK (rating >= 0 AND rating <= 10),
   room_id INT NOT NULL,
   guest_id INT NOT NULL,
   review_date TIMESTAMP NOT NULL,
   
   FOREIGN KEY (room_id)
       REFERENCES rooms (id),
   FOREIGN KEY (guest_id)
       REFERENCES guests (id)
);
'''

reservations = '''
CREATE TABLE IF NOT EXISTS reservations(
   id serial PRIMARY KEY,
   room_id INT NOT NULL,
   guest_id INT NOT NULL,
   check_in_date TIMESTAMP NOT NULL,
   check_out_date TIMESTAMP CHECK (check_out_date > check_in_date),
   paid BOOLEAN,

   FOREIGN KEY (room_id)
       REFERENCES rooms (id),
   FOREIGN KEY (guest_id)
       REFERENCES guests (id)
);'''

In [4]:
for query in (hosts, guests, rooms, host_reviews, guest_reviews, reservations):
    conn.execute(query)

In [5]:
hosts_values = (
    (1, 'Ann', 'ann_maiko@gmail.com'),
    (2, 'Sergiy', '+38-096-666-22-56'),
    (3, 'Kate',  '0667779090'))

guests_values = (
    (1, 'Lesya', 'frankol2001@gmail.com'),
    (2, 'Viktor', 'v_a_bober@gmail.com'),
    (3, 'Anton', 'toha_kastet@gmail.com'),
    (4, 'Maryna Andriivna', '+380732284567'))

rooms_values = (
    (1, 1, 'country house near the mountains', 'Spend an unforgettable weekend in nature and comfort',
        2, 900.00, 'Ukraine, Ivano-Frankivsk region, Tatariv', False),
    (2, 2, 'Modern and quiet apartment', 'Spacious apartment for those who like to be alone',
        1,  1297.30, 'Ukraine, Lviv',True),
    (3, 2, 'Сozy apartment in the historic center', 'All the most interesting things within 10 minutes', 
        2, 1999.99, 'Ukraine, Lviv', True))

reservations_values = (
    (1, 2, 1, '21-01-2022', '24-01-2022', True),
    (2, 1, 2, '09-08-2022', '13-08-2022', True),
    (3, 2, 3, '25-11-2022', '26-11-2022', False),
    (4, 1, 4, '20-01-2023', '23-01-2023', False),
    (5, 2, 1, '22-01-2023', '26-01-2023', True))

guest_reviews_values = (
    (1, 'I liked it!', 9, 2, 1, '01-02-2022'), 
    (2, 'There are a lot of mosquitoes and stuffy, but the rest is fine', 7, 1, 2, '01-09-2022'), 
    (3, '.', 3, 2, 3,'11-12-2022'))

host_reviews_values = (
    (1, 'They did not leave a mess behind and paid in advance', 9, 2, 2, 1, '26-01-2022'),
    (2, '''The terrible guest, smashed up the apartment, 
         left bottles and cigarettes behind, spat all over
         the apartment and did not pay!!!!''', 0, 2, 2, 3, '26-11-2022'),
    (3, 'They`re a nice young couple, but they`re too shy.', 8, 1, 1, 2,'20-08-2022'))

In [6]:
def insert(table, values):
    q = ('?, ' * len(values[0])).rstrip(', ')
    query = f'''
            INSERT INTO {table} VALUES ({q})
            '''
    for val in values:
        conn.execute(query, val)

In [7]:
tables = ('hosts', 'guests', 'rooms', 'host_reviews', 'guest_reviews', 'reservations')
values = (hosts_values, 
          guests_values, 
          rooms_values, 
          host_reviews_values, 
          guest_reviews_values, 
          reservations_values)


In [8]:
for t, v in zip(tables, values):
    insert(t, v)

for t in tables:
    conn.execute(f'''drop table {t}
    ''')

1. Write SQL queries for table creation for data model that you crated for prev homework (AirBnb model),
2. Write 3 rows (using INSERT queries) for each table in the data model,
3. Create next analytic queries:
        1. Find a user who had biggest amount of reservation. 
           Return user name and user_id,
        2. (Optional) Find a host who earned biggest amount of money for the last month. 
           Return host name and host_id,
        3. (Optional) Find a host with best average rating.
           Return host name and host_id

In [9]:
query1 = '''
WITH max_guest AS (
SELECT guest_id, COUNT(guest_id)
FROM reservations
GROUP BY guest_id
ORDER BY COUNT(guest_id) DESC
LIMIT 1)

SELECT guest_id, name
FROM guests
JOIN max_guest ON max_guest.guest_id = guests.id;
'''

task1 = conn.execute(query1)
task1.fetchall()

[(1, 'Lesya')]

In [13]:
query2postgres = '''
WITH room_income AS(
SELECT EXTRACT(DAY FROM check_out_date - check_in_date) * rooms.price AS earned_money, rooms.host_id
FROM reservations
JOIN rooms ON rooms.id = reservations.room_id
WHERE paid = True)

SELECT id, name
FROM (SELECT SUM(room_income.earned_money), hosts.id, hosts.name
FROM room_income
JOIN hosts ON hosts.id = room_income.host_id
GROUP BY hosts.id, hosts.name
ORDER BY SUM(earned_money) DESC
LIMIT 1) AS max_host_income
;
'''

query2 = '''
WITH room_income AS(
SELECT (check_out_date - check_in_date) * rooms.price AS earned_money, rooms.host_id
FROM reservations
JOIN rooms ON rooms.id = reservations.room_id
WHERE paid = True)

SELECT host_id, name
FROM (SELECT host_id, ROUND(SUM(earned_money), 2), hosts.name
FROM room_income
JOIN hosts ON hosts.id = room_income.host_id
GROUP BY host_id
ORDER BY SUM(earned_money) DESC
LIMIT 1)
;
'''

task2 = conn.execute(query2)
task2.fetchall()

[(2, 'Sergiy')]

In [12]:
query3 = '''
WITH best_host AS (
SELECT room_id, AVG(rating) AS avg_rating, rooms.host_id
FROM guest_reviews
JOIN rooms ON rooms.id = guest_reviews.room_id
GROUP BY room_id, rooms.host_id
ORDER BY AVG(rating) DESC
LIMIT 1)

SELECT id, name
FROM hosts
JOIN best_host ON hosts.id = best_host.host_id
;
'''

task3 = conn.execute(query3)
task3.fetchall()

[(1, 'Ann')]